In [1]:
!pip install transformers
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 152.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 164.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 133.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 86.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch as t

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
pretrained_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [3]:
from datasets import load_dataset
ds = load_dataset('stas/openwebtext-10k')
dataset = ds['train']['text']

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


/home/ubuntu/.local/lib/python3.8/site-packages/datasets/download/download_manager.py:527: FutureWarning: 'num_proc' was deprecated in version 2.6.2 and will be removed in 3.0.0. Pass `DownloadConfig(num_proc=<num_proc>)` to the initializer instead.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset openwebtext-10k downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/stas___openwebtext-10k/plain_text/1.0.0/3a8df094c671b4cb63ed0b41f40fb3bd855e9ce2e3765e5df50abcdfb5ec144b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
pretrained_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [5]:
def init_layer(layer: t.nn.Module):
    if isinstance(layer, t.nn.Embedding) or isinstance(layer, t.nn.Linear):
        layer.weight.data.normal_(0, 0.02)

In [6]:
class GPTBlock(t.nn.Module):
    def __init__(self, hidden_size = 768, context_length = 1024, dim_size = 3072, p_dropout = 0.1, n_heads = 12):
        super().__init__()
        self.ln_init = t.nn.LayerNorm(hidden_size)
        self.attn = t.nn.MultiheadAttention(hidden_size, n_heads, p_dropout, batch_first = True)
        mask = (t.triu(t.ones(context_length, context_length)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        self.attn_mask = t.nn.Parameter(mask, requires_grad = False)
        self.ln_intermediate = t.nn.LayerNorm(hidden_size)
        self.nn1 = t.nn.Linear(hidden_size, dim_size)
        self.nn2 = t.nn.Linear(dim_size, hidden_size)
        self.gelu = t.nn.GELU()
        self.dropout = t.nn.Dropout(p_dropout)
    
    def forward(self, x):
        resid_0 = x
        x = self.ln_init(x)
        x, _ = self.attn(x, x, x, attn_mask = self.attn_mask, need_weights = False)
        x = self.ln_intermediate(x + resid_0)
        resid_1 = x
        x = self.nn1(x)
        x = self.nn2(x)
        x = self.gelu(x)
        return self.dropout(x + resid_1)

In [7]:
class SimpleGPT2(t.nn.Module):
    def __init__(self, n_blocks = 1, vocab_size = 50257, context_length = 1024, hidden_size = 768, p_dropout = 0.1):
        super().__init__()
        self.wte = t.nn.Embedding(vocab_size, hidden_size)
        self.wpe = t.nn.Embedding(context_length, hidden_size)
        self.pe_matrix = t.nn.Parameter(t.arange(0, context_length).unsqueeze(0), requires_grad = False)
        self.dropout = t.nn.Dropout(p_dropout)
        self.gpt_blocks = t.nn.ModuleList([GPTBlock() for _ in range(n_blocks)])
        self.layernorm = t.nn.LayerNorm(hidden_size)
        self.final = t.nn.Linear(hidden_size, vocab_size)

        for layer in [self.wte, self.wpe, self.final]:
            init_layer(layer)
    
    def forward(self, input_ids: t.Tensor, attention_mask = t.Tensor):
        x = input_ids
        n, seq_len = x.shape
        hidden = self.wte(x) + self.wpe(self.pe_matrix.expand(n, -1))
        hidden = self.dropout(hidden)
        for gpt_block in self.gpt_blocks:
            hidden = gpt_block(hidden)
        hidden = self.layernorm(hidden)
        return self.final(hidden)




In [9]:
simpleGPT2 = t.load('simpleGPT_10_epochs.pkl')

# Run model on a few truncated samples ... works!

encoded_input = tokenizer(dataset[0:1], return_tensors='pt', padding='max_length', truncation=True).to(device)
print(encoded_input['attention_mask'].shape, encoded_input['attention_mask'].sum())
logits = simpleGPT2(**encoded_input)
print(logits.shape)

torch.Size([1, 1024]) tensor(1024, device='cuda:0')
torch.Size([1, 1024, 50257])


In [10]:
# How many parameters?
print(sum((p.numel() if p.requires_grad else 0 for p in simpleGPT2.parameters())))

120560209


In [11]:
encoded_input_alt = tokenizer(dataset[0][:100], return_tensors='pt', padding='max_length', truncation=True).to(device)
print(encoded_input_alt['attention_mask'].shape, encoded_input_alt['attention_mask'].sum())

torch.Size([1, 1024]) tensor(21, device='cuda:0')


In [12]:
def greedy_sampling(logits):
  return logits.argmax()

def test_model(model, text = "Replace me by any text you'd like.", steps = 100, sampling = greedy_sampling, is_hf = False):
    eos_token = "<|endoftext|>"
    prompt = text
    print("Starting prompt: " + prompt)

    for i in range(steps):
        encoded_input = tokenizer([prompt], return_tensors="pt", padding='max_length').to(device)
        last_input_idx = encoded_input['attention_mask'][0].sum() - 1
        if is_hf:
            logits = model(**encoded_input).logits[0, last_input_idx]
        else:
            logits = model(**encoded_input)[0, last_input_idx]
        next_token = sampling(logits)
        next_string = tokenizer.decode(next_token)
        if next_string == eos_token:
            break
        prompt = prompt + next_string
    print("Current generation: " + prompt)

In [13]:
def top_k_sampling(k):


      def top_sampling(logits):
          probs = t.nn.functional.softmax(logits)
          values, indices = t.topk(probs, k)
          index = values.multinomial(num_samples = 1, replacement = True)
          return indices[index]
      
      return top_sampling

# Our model generates English, but not really coherent generations.
test_model(simpleGPT2, text = "Mary is the greatest. Or is she?", steps = 100, sampling = top_k_sampling(100))
test_model(pretrained_model, text = "Mary is the greatest. Or is she?", is_hf = True, steps = 100, sampling = top_k_sampling(100))

Starting prompt: Mary is the greatest. Or is she?


<ipython-input-13-b49424ce377b>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = t.nn.functional.softmax(logits)
<ipython-input-13-b49424ce377b>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = t.nn.functional.softmax(logits)


Current generation: Mary is the greatest. Or is she? There (13 PM

 #568)
The National Service) Posted an entire protected, but a great end of the U.S. Senate GRC.

 is running their life.

ed by his work the one day when helled, but he� has seen as soon.com We take him to be made a trip. Please take away from the president to do in-up to make it can bet that, the area.

, and if there,
Starting prompt: Mary is the greatest. Or is she?
Current generation: Mary is the greatest. Or is she?


J. R.R.: (We go round the corner with three o'clock.) Uh, she is. Then what's your name-


GARTH: I am an old man of five senses.


J. R.R.: I was born in Birmingham, Massachusetts.


GARTH: Where are my ancestors!?


J. R.R.:... I mean the family of my grandmother.


GARTH: All right!


In [14]:
def loss_fn(logits, encoded_input):
    # logits: n x seq x d
    # true_tokens: n x seq
    # attention_mask = n x seq
    true_tokens = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']
    valid_samples_mask = attention_mask[:, 1:].reshape(-1).bool()
    n, seq, d  = logits.shape
    return t.nn.functional.cross_entropy(logits[:, :-1, :].reshape(-1, d)[valid_samples_mask, :], true_tokens[:, 1:].flatten()[valid_samples_mask]), valid_samples_mask.sum()

def compute_dataset_loss(dataset, model, tokenizer):
    loss = 0
    samples = 0
    with t.no_grad():
      n = len(dataset)
      batch_size = 10
      batches = n // batch_size
      for i in range(batches):
          print(i, batch_size, loss, samples)
          batch = dataset[i:i+batch_size]
          encoded_input = tokenizer(batch, return_tensors='pt', padding='max_length', truncation=True).to(device)
          logits = model(**encoded_input)
          # Find true labels and compute loss
          ce_loss, valid_samples = loss_fn(logits, encoded_input)
          loss = (loss * samples + ce_loss * valid_samples ) / (samples + valid_samples)
          samples = samples + valid_samples
    return loss, samples

# Compute loss of the pre-trained model on the truncated dataset
print(compute_dataset_loss(dataset[:100], simpleGPT2, tokenizer))

# Initial loss is ~5.8, better than 6, but still high.

0 10 0 0
1 10 tensor(5.8174, device='cuda:0') tensor(6508, device='cuda:0')
2 10 tensor(5.8347, device='cuda:0') tensor(12338, device='cuda:0')
3 10 tensor(5.8356, device='cuda:0') tensor(18466, device='cuda:0')
4 10 tensor(5.8111, device='cuda:0') tensor(24853, device='cuda:0')
5 10 tensor(5.8118, device='cuda:0') tensor(30959, device='cuda:0')
6 10 tensor(5.8051, device='cuda:0') tensor(37244, device='cuda:0')
7 10 tensor(5.7859, device='cuda:0') tensor(43529, device='cuda:0')
8 10 tensor(5.7658, device='cuda:0') tensor(50599, device='cuda:0')
9 10 tensor(5.7544, device='cuda:0') tensor(57171, device='cuda:0')
(tensor(5.7420, device='cuda:0'), tensor(63899, device='cuda:0'))


In [15]:
def compute_val_dataset_loss(dataset, model, tokenizer, val_frac = 0.2):
    n = len(dataset)
    val_size = int(n * val_frac)
    return compute_dataset_loss(dataset[-val_size:], model, tokenizer)
  
# Compute validation loss
# print(compute_val_dataset_loss(dataset, 0.1))

In [17]:
# Fine-tune the model on a subset of training set, and then evaluate on val set
import random

def train_model(dataset, optimizer, epochs, model, tokenizer):
    loss = 0
    samples = 0
    n = len(dataset)
    batch_size = 2
    batches = n // batch_size
    print_interval = batches // 20

    scheduler = OneCycleLR(optimizer, max_lr = 2.5e-4, total_steps = epochs * batches, pct_start = 0.2)

    for epoch in range(epochs):
        random.shuffle(dataset)
        print("Starting epoch: ", epoch)
        for i in range(batches):
            if i % print_interval == 0:
                print(i, i/float(batches), batch_size, loss, samples)

            optimizer.zero_grad()

            batch = dataset[i:i+batch_size]
            for i, e in enumerate(batch):
                offset = random.randint(0, max(len(e) - 2048, 0))
                batch[i] = e[offset:]
            encoded_input = tokenizer(batch, return_tensors='pt', padding='max_length', truncation=True).to(device)
            logits = model(**encoded_input)

            # Find true labels and compute loss
            ce_loss, valid_samples = loss_fn(logits, encoded_input)
            loss = (loss * samples + ce_loss * valid_samples ) / (samples + valid_samples)
            samples = samples + valid_samples

            # Backprop
            ce_loss.backward()
            optimizer.step()
            scheduler.step()

    return loss, samples

epochs = 10
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR
import random

lrs = [5e-5, 5e-4, 1e-5, 2e-5]

for i in range(4):
  start_idx = 2000*i
  end_idx = 2000*(i+1)
  optimizer = Adam(simpleGPT2.parameters(), lr = lrs[-1])
  print(train_model(dataset[start_idx:end_idx], optimizer, epochs, simpleGPT2, tokenizer))
# Loss reaches 6.3 after 3 epochs


Starting epoch:  0
0 0.0 2 0 0
50 0.05 2 tensor(5.8601, device='cuda:0', grad_fn=<DivBackward0>) tensor(64444, device='cuda:0')
100 0.1 2 tensor(5.8409, device='cuda:0', grad_fn=<DivBackward0>) tensor(121341, device='cuda:0')
150 0.15 2 tensor(5.8481, device='cuda:0', grad_fn=<DivBackward0>) tensor(175105, device='cuda:0')
200 0.2 2 tensor(5.8747, device='cuda:0', grad_fn=<DivBackward0>) tensor(240812, device='cuda:0')
250 0.25 2 tensor(5.8599, device='cuda:0', grad_fn=<DivBackward0>) tensor(301530, device='cuda:0')
300 0.3 2 tensor(5.8462, device='cuda:0', grad_fn=<DivBackward0>) tensor(357536, device='cuda:0')
350 0.35 2 tensor(5.8476, device='cuda:0', grad_fn=<DivBackward0>) tensor(414450, device='cuda:0')
400 0.4 2 tensor(5.8512, device='cuda:0', grad_fn=<DivBackward0>) tensor(474074, device='cuda:0')
450 0.45 2 tensor(5.8464, device='cuda:0', grad_fn=<DivBackward0>) tensor(533749, device='cuda:0')
500 0.5 2 tensor(5.8226, device='cuda:0', grad_fn=<DivBackward0>) tensor(593780, dev

In [ ]:
test_model(simpleGPT2, text = "Mary is the greatest. Or is she?", steps = 100, sampling = top_k_sampling(100))
test_model(pretrained_model, text = "Mary is the greatest. Or is she?", is_hf = True, steps = 100, sampling = top_k_sampling(100))
print(compute_val_dataset_loss(dataset, simpleGPT2, tokenizer, 0.1))

# Generations seem to be getting better, or maybe that's just my imagination.
# Loss on val set is around 6.05, although training loss has broken 6.0

Starting prompt: Mary is the greatest. Or is she?


<ipython-input-13-b49424ce377b>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = t.nn.functional.softmax(logits)


Current generation: Mary is the greatest. Or is she? There's been some understanding that people

 than most beast for someone out there is a lot of thinking about to be a place? But she goes to stay to learn more people that Woolinating by a female, the woman who do you've got old. He doesn't say. When somebody who aren�isting mistake alone, no way of work to say it down anymore this fact that we knew the kids out, for the faith, and feel a great influence an atheist. This is a free
Starting prompt: Mary is the greatest. Or is she?
Current generation: Mary is the greatest. Or is she? Even the other children in her family, who live in the White House, are born with a great deal of power. [They] don't want anyone else to be seen. They are in awe. They have learned from history, by the Bible."


Reuben the Great is a member of the Boy Scouts of America, a group that "encourages and fosters men, women and children to care for the American spirit." Unlike most of his peers, he does not con

In [22]:
t.save(simpleGPT2, 'simpleGPT_10_epochs.pkl')